In [3]:
import pandas as pd
import numpy as np
import anndata
import json
import os
import scanpy as sc
from sklearn.model_selection import train_test_split

## Data download

In [4]:

# Define data paths
data_dir = "data_input"
os.makedirs(data_dir, exist_ok=True)

pancreas_adata_path = os.path.join(data_dir, "pancreas_full.h5ad")
train_path = os.path.join(data_dir, "pancreas_train.h5ad")
valid_path = os.path.join(data_dir, "pancreas_valid.h5ad")
test_path  = os.path.join(data_dir, "pancreas_test.h5ad")

# Download if missing, otherwise load from local file
pancreas_adata = sc.read(
    pancreas_adata_path,
    backup_url="https://figshare.com/ndownloader/files/24539828",
)

# Split dataset by technology: keep smartseq2/celseq2 as held-out test
query_mask = pancreas_adata.obs["tech"].isin(["smartseq2", "celseq2"]).to_numpy()
pancreas_no_test = pancreas_adata[~query_mask].copy()
pancreas_test    = pancreas_adata[ query_mask].copy()

# 80/20 train/valid split on the remaining data, stratified by technology
y = pancreas_no_test.obs["tech"].astype("category")
indices = np.arange(pancreas_no_test.n_obs)

idx_train, idx_valid = train_test_split(
    indices,
    test_size=0.20,
    train_size=0.80,
    random_state=42,
    shuffle=True,
    stratify=y  # stratify by technology
)

pancreas_train = pancreas_no_test[idx_train].copy()
pancreas_valid = pancreas_no_test[idx_valid].copy()

# Save splits
pancreas_train.write(train_path)
pancreas_valid.write(valid_path)
pancreas_test.write(test_path)

print(
    f"Train: {pancreas_train.n_obs} cells | "
    f"Valid: {pancreas_valid.n_obs} cells | "
    f"Test: {pancreas_test.n_obs} cells"
)

# Print counts per technology
print("\nCells per technology:")
for name, ad in [("Train", pancreas_train),
                 ("Valid", pancreas_valid),
                 ("Test", pancreas_test)]:
    counts = ad.obs["tech"].value_counts().sort_index()
    print(f"\n{name} split:")
    for tech, n in counts.items():
        print(f"  {tech}: {n}")

# --- Cleanup: delete the original full dataset file ---
del pancreas_adata  # drop reference to ensure no open handle
try:
    if os.path.exists(pancreas_adata_path):
        os.remove(pancreas_adata_path)
        print(f"Deleted '{pancreas_adata_path}'")
except Exception as e:
    print(f"[WARN] Could not delete '{pancreas_adata_path}': {e}")


# --- Save full gene list to JSON ---
all_genes = pancreas_train.var_names.tolist()

genes_json_path = os.path.join("data_input", "all_genes_list.json")
os.makedirs("data_input", exist_ok=True)

with open(genes_json_path, "w") as f:
    json.dump(all_genes, f, indent=2)

print(f"Saved {len(all_genes)} genes to {genes_json_path}")

Train: 9362 cells | Valid: 2341 cells | Test: 4679 cells

Cells per technology:

Train split:
  celseq: 803
  fluidigmc1: 510
  inDrop1: 1550
  inDrop2: 1379
  inDrop3: 2884
  inDrop4: 1042
  smarter: 1194

Valid split:
  celseq: 201
  fluidigmc1: 128
  inDrop1: 387
  inDrop2: 345
  inDrop3: 721
  inDrop4: 261
  smarter: 298

Test split:
  celseq2: 2285
  smartseq2: 2394
Deleted 'data_input/pancreas_full.h5ad'
Saved 19093 genes to data_input/all_genes_list.json


## Data inspection

In [5]:
# Read the data
adata = anndata.read_h5ad("./data_input/pancreas_train.h5ad")

In [6]:
# Display the AnnData object summary
print("AnnData object summary:")
print(adata)

# Display the first few rows of the observation metadata
print("\nFirst 5 rows of adata.obs:")
print(adata.obs.head())

# Display available layers
print("\nAvailable layers in adata:")
print(adata.layers.keys())

# Display the first 5x5 block of the counts layer (if it exists)
print("\nFirst 5x5 of counts layer:")
print(pd.DataFrame(adata.layers["counts"][:5, :5], 
                    columns=adata.var_names[:5], 
                    index=adata.obs_names[:5]))

AnnData object summary:
AnnData object with n_obs × n_vars = 9362 × 19093
    obs: 'tech', 'celltype', 'size_factors'
    layers: 'counts'

First 5 rows of adata.obs:
                                   tech celltype  size_factors
3rd-C86_S85                  fluidigmc1    delta      5.060723
human3_lib4.final_cell_0804     inDrop3    alpha      0.010361
human3_lib4.final_cell_0815     inDrop3     beta      0.011553
Sample_163                      smarter     beta      1.000000
human3_lib1.final_cell_0737     inDrop3    alpha      0.014493

Available layers in adata:
KeysView(Layers with keys: counts)

First 5x5 of counts layer:
                             A1BG  A1CF  A2M  A2ML1  A4GALT
3rd-C86_S85                  14.7  11.0  0.0    3.0     0.0
human3_lib4.final_cell_0804   0.0   0.0  0.0    0.0     0.0
human3_lib4.final_cell_0815   0.0   0.0  0.0    0.0     0.0
Sample_163                    0.0   0.0  0.0    0.0     0.0
human3_lib1.final_cell_0737   0.0   1.0  0.0    0.0     0.0


In [7]:
# 1. How many unique technologies are present, and what are their names?

techs = adata.obs['tech'].unique()
print(f"Number of unique technologies: {len(techs)}")
print("Technologies:", ", ".join(techs))

Number of unique technologies: 7
Technologies: fluidigmc1, inDrop3, smarter, inDrop1, celseq, inDrop4, inDrop2


In [8]:
# 2. How many samples (cells) belong to each technology?

tech_counts = adata.obs['tech'].value_counts()
print("\nNumber of samples (cells) per technology:")
print(tech_counts.to_string())


Number of samples (cells) per technology:
tech
inDrop3       2884
inDrop1       1550
inDrop2       1379
smarter       1194
inDrop4       1042
celseq         803
fluidigmc1     510


In [9]:
# 3. What is the total number of genes measured in the dataset?

num_genes = adata.shape[1]
print(f"\nTotal number of genes measured: {num_genes}")


Total number of genes measured: 19093


In [10]:
# 4. What is the total number of samples (cells) in the dataset?

num_samples = adata.shape[0]
print(f"\nTotal number of samples (cells): {num_samples}")


Total number of samples (cells): 9362


## Variance analysis

In [11]:
# 5. For each technology, calculate the variance of each gene across all cells.

data = adata.layers["counts"]

techs = adata.obs['tech'].unique()
gene_names = adata.var_names

var_df = pd.DataFrame(index=gene_names)

for tech in techs:
    idx = adata.obs['tech'] == tech
    tech_data = data[idx, :]
    var_df[tech] = np.var(tech_data, axis=0)

print("\nVariance DataFrame (first 5 genes):")
print(var_df.head())


Variance DataFrame (first 5 genes):
          fluidigmc1   inDrop3     smarter   inDrop1    celseq   inDrop4  \
A1BG      835.186768  0.002079   99.446968  0.014618  0.710532  0.003824   
A1CF    48044.792969  0.226555  418.187988  0.337120  0.802834  0.544838   
A2M     80713.156250  0.042076    0.415398  0.217431  0.077846  0.163033   
A2ML1      27.619577  0.000000    0.081731  0.000000  0.314199  0.000000   
A4GALT     92.523102  0.153656    0.111626  0.070562  0.057321  0.150387   

         inDrop2  
A1BG    0.004332  
A1CF    0.479885  
A2M     0.281788  
A2ML1   0.000000  
A4GALT  0.097811  


In [12]:
# 6. Compute a weighted average of these variances for each gene, using the number of cells per technology as weights.

weights = np.array([tech_counts[tech] for tech in var_df.columns])  # using tech_counts from point 2.
weighted_vars = (var_df.values * weights).sum(axis=1) / weights.sum()

# Store in a DataFrame with gene name and weighted variance
weighted_var_df = pd.DataFrame({
    "gene": gene_names,
    "weighted_variance": weighted_vars
}).set_index("gene")

print("\nWeighted variance DataFrame (first 5 genes):")
print(weighted_var_df.head())


Weighted variance DataFrame (first 5 genes):
        weighted_variance
gene                     
A1BG            58.245471
A1CF          2670.926186
A2M           4397.060993
A2ML1            1.541965
A4GALT           5.149561


In [13]:
# 7. Save a list containing the top 2000 genes
top2000_genes = (
    weighted_var_df.sort_values("weighted_variance", ascending=False)
    .head(2000)
    .index.tolist()
)

# Save to a JSON file
with open("./data_output/top2000_genes_centralized.json", "w") as f:
    json.dump(top2000_genes, f, indent=2)